In [30]:
# Importazione delle librerie necessarie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Impostazione dello stile grafico
sns.set(style="whitegrid")

In [31]:
# Caricamento del dataset
df = pd.read_csv('EnergyConsumption.csv')

# Visualizzazione delle prime righe
df.head()

,Datetime,AEP_MW
0,2004-12-31 01:00:00,13478.0
1,2004-12-31 02:00:00,12865.0
2,2004-12-31 03:00:00,12577.0
3,2004-12-31 04:00:00,12517.0
4,2004-12-31 05:00:00,12670.0


In [32]:
df.info

<bound method DataFrame.info of                    Datetime   AEP_MW
0       2004-12-31 01:00:00  13478.0
1       2004-12-31 02:00:00  12865.0
2       2004-12-31 03:00:00  12577.0
3       2004-12-31 04:00:00  12517.0
4       2004-12-31 05:00:00  12670.0
...                     ...      ...
121268  2018-01-01 20:00:00  21089.0
121269  2018-01-01 21:00:00  20999.0
121270  2018-01-01 22:00:00  20820.0
121271  2018-01-01 23:00:00  20415.0
121272  2018-01-02 00:00:00  19993.0

[121273 rows x 2 columns]>

# 1a Parte: Cleaning e Resample dei Dati

Convertiamo la data da stringa in DateTime

In [33]:
df['Datetime'] = pd.to_datetime(df['Datetime'], format="%Y-%m-%d %H:%M:%S")

In [34]:
# Mettiamo il DateTame come indice dell'asse X e mi assicuro di ordinarli dal più vecchio al più recente
df = df.set_index('Datetime')
df = df.sort_index()

# Verifica
print(f"Range temporale: dal {df.index.min()} al {df.index.max()}")
df.head()

Range temporale: dal 2004-10-01 01:00:00 al 2018-08-03 00:00:00


,AEP_MW
Datetime,
2004-10-01 01:00:00,12379.0
2004-10-01 02:00:00,11935.0
2004-10-01 03:00:00,11692.0
2004-10-01 04:00:00,11597.0
2004-10-01 05:00:00,11681.0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 121273 entries, 2004-10-01 01:00:00 to 2018-08-03 00:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   AEP_MW  121273 non-null  float64
dtypes: float64(1)
memory usage: 1.9 MB


### Rimozione di duplicati
I duplicati sono frequenti a causa del cambio dell'ora legale o errori dei sensori.
Manteniamo la prima rilevazione per garantire l'unicità dell'indice temporale.

In [36]:
# Verifico se ci sono dei duplicati
duplicati = df.index.duplicated().sum()
print(f"Numero di orari duplicati trovati: {duplicati}")

# Rimozione Duplicati
# Mantengo solo la prima occorrenza con 'first'.
df = df[~df.index.duplicated(keep='first')]

print("Duplicati rimossi.")
print("Duplicati attuali:", df.index.duplicated().sum())

Numero di orari duplicati trovati: 4
Duplicati rimossi.
Duplicati attuali: 0


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 121269 entries, 2004-10-01 01:00:00 to 2018-08-03 00:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   AEP_MW  121269 non-null  float64
dtypes: float64(1)
memory usage: 5.9 MB


### Cerchiamo se ci sono valori mancanti

I valori mancanti (NaN) possono derivare da blackout del sensore o errori di trasmissione. Non possiamo accettare valori vuoti in input per la parte di Machine Learning. Verifichiamo la loro presenza per decidere se imputarli (stimarli) o rimuovere le righe corrotte."

In [38]:
# Contiamo i valori mancanti (NaN)
missing = df.isna().sum()
print("Valori mancanti per colonna:\n", missing)

# Se ce ne sono pochi, li possiamo eliminare (dropna) o riempire (fillna)
# In questo dataset, se sono pochi, li eliminiamo per pulizia.
if missing['AEP_MW'] > 0:
    df = df.dropna()
    print("Valori mancanti eliminati.")
else:
    print("Il dataset è completo, nessuna azione necessaria.")

Valori mancanti per colonna:
 AEP_MW    0
dtype: int64
Il dataset è completo, nessuna azione necessaria.


### Fine 1a Parte: Cleaning e Resample dei Dati
Ora abbiamo un Dataset Temporale, Ordinato, Senza Doppioni e Senza Mancanze.